# Library Setting

In [1]:
from tqdm import tqdm
tqdm.pandas()

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

import gc
gc.collect()

0

<br></br>

# Configuration

In [2]:
class CFG:
    SEED = 0
    
    SUBSET_DEPTH = 3
    INTERACTION = False
    FS_ALPHA = 0.01
    
    N_SPLITS = 5
    TARGET_TRANSFORMATION = True
    
    LR = 0.03
    EPOCHS = 30000
    ES = 300
    XGB_LR = 0.3     # default
    XGB_EPOCHS = 1000 # default
    XGB_ES = 10

<br></br>

# Data

## Data Load

In [13]:
train_df = pd.read_csv('./data/train.csv')
test_df  = pd.read_csv('./data/test.csv')

In [14]:
train_df.shape, test_df.shape

((57920, 15), (14480, 14))

In [15]:
train_df.head()

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격
0,TRAIN_00000,2018,2014,skoda,fabia,KAT,SLA,85231,999,0,0,1,0,0,51.74
1,TRAIN_00001,2010,2006,toyota,auris,RKO,SWI,135000,1598,0,0,1,0,0,41.47
2,TRAIN_00002,2002,2002,mercedes-benz,clk-klasa,GNI,WIE,255223,1796,0,0,1,0,0,17.81
3,TRAIN_00003,2006,2001,nissan,x-trail,EHX,WIE,238000,2184,0,1,0,0,0,18.20
4,TRAIN_00004,2007,2007,fiat,bravo,OSW,MAL,251000,1910,0,1,0,0,0,17.55


<br>

## Target Transformation

In [16]:
if CFG.TARGET_TRANSFORMATION:
    train_df['가격'] = np.log(train_df['가격'])

<br>

## Resetting Columns Type

In [18]:
class TypeResetting:
    def __init__(self):
        self.cat_features = ['브랜드','차량모델명','판매도시','판매구역','생산년도','모델출시년도']
        self.seg_features = []
        
    def add_categorical_features(self,cat_features):
        self.cat_features += cat_features
        
    def delete_categorical_features(self,cat_features):
        self.cat_features = [col for col in self.cat_features if col not in cat_features]
        
    def add_segment_features(self,segment_features):
        self.seg_features = ['segment']
        self.cat_features = [col for col in self.cat_features if col not in segment_features]
        
    def fit(self,data):
        if (len(self.seg_features)>0) & ('segment' not in data.columns):
            raise ValueError("segment column name must be 'segment'")
        self.target_feature = ['가격']
        self.unuse_features = ['ID']
        self.dummy_features = ['압축천연가스(CNG)','액화석유가스(LPG)','경유','가솔린','하이브리드']
        self.num_features   = [col for col in data.columns
                               if col not in self.target_feature+self.unuse_features+self.dummy_features+self.cat_features+self.seg_features]
        
    def transform(self,data):
        d = data.copy()
        for col in self.dummy_features:
            if d[col].dtypes!=int:
                d[col] = d[col].astype(int)
        for col in self.cat_features:
            if d[col].dtypes!=object:
                d[col] = d[col].astype(str)
        for col in self.num_features:
            if d[col].dtypes!=float:
                d[col] = d[col].astype(float)
        for col in self.seg_features:
            if d[col].dtypes!=object:
                d[col] = d[col].astype(str)
        for col in self.unuse_features:
            if col in d.columns:
                d.drop(col,axis=1,inplace=True)
        return d
    
    def fit_transform(self,data):
        self.fit(data)
        return self.transform(data)
    
    def get_feature_type(self):
        globals()['target_feature'] = self.target_feature
        globals()['unuse_features'] = self.unuse_features
        globals()['dummy_features'] = self.dummy_features
        globals()['cat_features']   = self.cat_features
        globals()['num_features']   = self.num_features

In [19]:
type_resetor = TypeResetting()
type_resetor.fit(train_df)
type_resetor.get_feature_type()

train_df2 = type_resetor.transform(train_df)
test_df2  = type_resetor.transform(test_df)

In [20]:
import warnings
warnings.simplefilter("always")

def check_only_oneside(train,test,cat_features):
    not_test_only_features = []
    for iter,col in enumerate(cat_features):
        print('[{}/{}] {}'.format(iter+1,len(cat_features),col))
        
        only_train = list(set(train[col].unique())-set(test[col].unique()))
        only_test  = list(set(test[col].unique())-set(train[col].unique()))
        print(' - Only Train:',len(only_train))
        print(' - Only Test :',len(only_test))
        if len(only_test)>0:
            print('******Warning******')
        else:
            not_test_only_features.append(col)
        print('')
    return not_test_only_features

In [21]:
# 브랜드, 차량모델명, 판매구역, 모델출시년도
not_test_only_features = check_only_oneside(train_df2,test_df2,cat_features+dummy_features)
not_test_only_features = list(set(not_test_only_features)-set(dummy_features))

[1/11] 브랜드
 - Only Train: 0
 - Only Test : 0

[2/11] 차량모델명
 - Only Train: 2
 - Only Test : 0

[3/11] 판매도시
 - Only Train: 1750
 - Only Test : 300
******Warning******

[4/11] 판매구역
 - Only Train: 0
 - Only Test : 0

[5/11] 생산년도
 - Only Train: 3
 - Only Test : 1
******Warning******

[6/11] 모델출시년도
 - Only Train: 0
 - Only Test : 0

[7/11] 압축천연가스(CNG)
 - Only Train: 0
 - Only Test : 0

[8/11] 액화석유가스(LPG)
 - Only Train: 0
 - Only Test : 0

[9/11] 경유
 - Only Train: 0
 - Only Test : 0

[10/11] 가솔린
 - Only Train: 0
 - Only Test : 0

[11/11] 하이브리드
 - Only Train: 0
 - Only Test : 0



In [22]:
not_test_only_features

['브랜드', '차량모델명', '판매구역', '모델출시년도']

<br></br>

# New Features

In [23]:
# pd.Series([str(round(int(year)/100,1)) for year in train_df6['생산년도']]).value_counts()

In [24]:
train_df.head()

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격
0,TRAIN_00000,2018,2014,skoda,fabia,KAT,SLA,85231,999,0,0,1,0,0,3.946231
1,TRAIN_00001,2010,2006,toyota,auris,RKO,SWI,135000,1598,0,0,1,0,0,3.724970
2,TRAIN_00002,2002,2002,mercedes-benz,clk-klasa,GNI,WIE,255223,1796,0,0,1,0,0,2.879760
3,TRAIN_00003,2006,2001,nissan,x-trail,EHX,WIE,238000,2184,0,1,0,0,0,2.901422
4,TRAIN_00004,2007,2007,fiat,bravo,OSW,MAL,251000,1910,0,1,0,0,0,2.865054


In [25]:
from tqdm import tqdm
from itertools import chain, combinations
def all_subsets(ss):
    return list(chain(*map(lambda x: combinations(ss, x), range(0, len(ss)+1))))

class FeatureEngineering:
    def __init__(self):
        pass
    
    def _get_quantile(self,x,col):
        x = np.array(x).flatten()
        x = x[pd.notnull(x)]

        agg_df = pd.DataFrame(index=[0])
        for q in [0,25,50,75,100]:
            agg_df[f'{col}_Q{q}'] = np.quantile(x,q/100)

        return agg_df
    
    def _derived_features(self,data):
        d = data.copy()

        # (1) 모델출시년도에 생산된 차량인지
        d['출시년도생산여부'] = np.where(d['생산년도'].astype(int)==d['모델출시년도'].astype(int),1,0)

        # (2) 모델출시 이후에 몇년 지나서 생산됬는지
        d['출시이후생산년수'] = d['생산년도'].astype(int)-d['모델출시년도'].astype(int)

        # (3) 출시 이전에 생산되었는지
        d['출시이전생산여부'] = np.where(d['출시이후생산년수']<0,1,0)

        # (4) 브랜드의 국적 (구글링)
        d['브랜드국적'] = ['체코' if brand=='skoda' else
                        '일본' if brand in ['toyota','nissan','mazda','honda','mitsubishi'] else
                        '독일' if brand in ['mercedes-benz','audi','volkswagen','bmw','opel'] else
                        '이탈리아' if brand=='fiat' else
                        '프랑스' if brand in ['renault','citroen','peugeot'] else
                        '미국' if brand=='ford' else
                        '한국' if brand in ['kia','hyundai'] else
                        '스페인' if brand=='seat' else
                        '스웨덴' if brand=='volvo' else
                        np.nan for brand in d['브랜드']]

        # (5) 브랜드 국적의 대륙명
        d['브랜드대륙명'] = ['유럽' if country in ['체코','독일','이탈리아','프랑스','스페인','스웨덴'] else
                          '아시아' if country in ['일본','한국'] else
                          '아메리카' if country in ['미국'] else
                          np.nan for country in d['브랜드국적']]
        return d
    
    def fit(self,data,cat_features,subset_depth=1):
        assert '가격' in data.columns, \
            'Input data must be training dataset'
        assert len(cat_features)>=subset_depth, \
            'len(cat_features) >= subset_depth'
        
        self.cat_features = cat_features
        self.new_cat_features = ['출시년도생산여부','출시이후생산년수','출시이전생산여부','브랜드국적','브랜드대륙명']
        
        # (6) 카테고리 변수에 따른 가격의 Quantile값
        all_subset_list = all_subsets(cat_features)
        all_subset_list = [subset for subset in all_subset_list if (len(subset)<=subset_depth) & (len(subset)>=1)]
        
        self.agg_dict = {}
        for subset in tqdm(all_subset_list,desc=f'Get quantiles of target by categorical features (depth={subset_depth})'):
            subset = list(subset)
            subset_name = '_'.join(subset)
            agg_fn = data.groupby(subset)['가격'].apply(lambda x: self._get_quantile(x,subset_name)).reset_index()
            drop_cols = [col for col in agg_fn if col.find('level_')>=0]
            agg_fn.drop(columns=drop_cols,inplace=True)
            self.agg_dict[subset_name] = agg_fn
            
    def transform(self,data):
        data = self._derived_features(data)
        for key,agg_fn in self.agg_dict.items():
            data = pd.merge(data,agg_fn,how='left',on=key.split('_'))
        return data
    
    def fit_transform(self,data,cat_features,subset_depth=1):
        self.fit(data,cat_features,subset_depth)
        return self.transform(data)

In [26]:
fe = FeatureEngineering()
fe.fit(
    data=train_df2,
    cat_features=not_test_only_features, 
    subset_depth=CFG.SUBSET_DEPTH,
)
train_df3 = fe.transform(train_df2)
test_df3  = fe.transform(test_df2)

Get quantiles of target by categorical features (depth=3): 100%|██████████| 14/14 [00:28<00:00,  2.02s/it]


In [27]:
fe.new_cat_features

['출시년도생산여부', '출시이후생산년수', '출시이전생산여부', '브랜드국적', '브랜드대륙명']

In [28]:
type_resetor = TypeResetting()
type_resetor.add_categorical_features(fe.new_cat_features)
type_resetor.fit(train_df3)
type_resetor.get_feature_type()

train_df3 = type_resetor.transform(train_df3)
test_df3  = type_resetor.transform(test_df3)

In [29]:
print(train_df3.shape)
train_df3.head()

(57920, 89)


,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격,출시년도생산여부,출시이후생산년수,출시이전생산여부,브랜드국적,브랜드대륙명,브랜드_Q0,브랜드_Q25,브랜드_Q50,브랜드_Q75,브랜드_Q100,차량모델명_Q0,차량모델명_Q25,차량모델명_Q50,차량모델명_Q75,차량모델명_Q100,판매구역_Q0,판매구역_Q25,판매구역_Q50,판매구역_Q75,판매구역_Q100,모델출시년도_Q0,모델출시년도_Q25,모델출시년도_Q50,모델출시년도_Q75,모델출시년도_Q100,브랜드_차량모델명_Q0,브랜드_차량모델명_Q25,브랜드_차량모델명_Q50,브랜드_차량모델명_Q75,브랜드_차량모델명_Q100,브랜드_판매구역_Q0,브랜드_판매구역_Q25,브랜드_판매구역_Q50,브랜드_판매구역_Q75,브랜드_판매구역_Q100,브랜드_모델출시년도_Q0,브랜드_모델출시년도_Q25,브랜드_모델출시년도_Q50,브랜드_모델출시년도_Q75,브랜드_모델출시년도_Q100,차량모델명_판매구역_Q0,차량모델명_판매구역_Q25,차량모델명_판매구역_Q50,차량모델명_판매구역_Q75,차량모델명_판매구역_Q100,차량모델명_모델출시년도_Q0,차량모델명_모델출시년도_Q25,차량모델명_모델출시년도_Q50,차량모델명_모델출시년도_Q75,차량모델명_모델출시년도_Q100,판매구역_모델출시년도_Q0,판매구역_모델출시년도_Q25,판매구역_모델출시년도_Q50,판매구역_모델출시년도_Q75,판매구역_모델출시년도_Q100,브랜드_차량모델명_판매구역_Q0,브랜드_차량모델명_판매구역_Q25,브랜드_차량모델명_판매구역_Q50,브랜드_차량모델명_판매구역_Q75,브랜드_차량모델명_판매구역_Q100,브랜드_차량모델명_모델출시년도_Q0,브랜드_차량모델명_모델출시년도_Q25,브랜드_차량모델명_모델출시년도_Q50,브랜드_차량모델명_모델출시년도_Q75,브랜드_차량모델명_모델출시년도_Q100,브랜드_판매구역_모델출시년도_Q0,브랜드_판매구역_모델출시년도_Q25,브랜드_판매구역_모델출시년도_Q50,브랜드_판매구역_모델출시년도_Q75,브랜드_판매구역_모델출시년도_Q100,차량모델명_판매구역_모델출시년도_Q0,차량모델명_판매구역_모델출시년도_Q25,차량모델명_판매구역_모델출시년도_Q50,차량모델명_판매구역_모델출시년도_Q75,차량모델명_판매구역_모델출시년도_Q100
0,2018,2014,skoda,fabia,KAT,SLA,85231.0,999.0,0,0,1,0,0,3.946231,0,4,0,체코,유럽,0.727549,3.393501,4.057853,4.509072,5.049856,0.792993,2.933059,3.712352,4.065687,4.830312,0.262364,3.206803,3.773910,4.355041,5.049856,0.157004,4.019486,4.355041,4.643140,5.049856,0.792993,2.933059,3.712352,4.065687,4.830312,1.137833,3.477232,4.021236,4.525044,5.049856,2.554899,3.843102,4.011868,4.172385,4.772124,1.137833,2.995955,3.814851,4.046554,4.741361,2.554899,3.843102,4.011868,4.172385,4.772124,3.287655,3.984523,4.322807,4.617790,5.049856,1.137833,2.995955,3.814851,4.046554,4.741361,2.554899,3.843102,4.011868,4.172385,4.772124,3.520461,3.864403,3.962369,4.120439,4.639378,3.520461,3.864403,3.962369,4.120439,4.639378
1,2010,2006,toyota,auris,RKO,SWI,135000.0,1598.0,0,0,1,0,0,3.724970,0,4,0,일본,아시아,0.157004,3.353407,4.000034,4.435212,5.049856,2.687167,3.594569,4.112512,4.387075,4.761062,1.095273,3.113071,3.520461,4.066802,5.049856,1.358409,3.089678,3.348851,3.785779,5.049856,2.687167,3.594569,4.112512,4.387075,4.761062,2.134166,3.341912,3.843102,4.286479,5.049022,2.687167,3.253084,3.501043,3.785779,4.450503,2.865054,3.356793,3.986240,4.276820,4.691990,2.687167,3.201526,3.353407,3.594569,4.173156,2.277267,3.127414,3.353407,3.782821,4.826712,2.865054,3.356793,3.986240,4.276820,4.691990,2.687167,3.201526,3.353407,3.594569,4.173156,2.865054,3.318510,3.606977,3.932255,4.450503,2.865054,3.144356,3.341912,3.632507,4.021236
2,2002,2002,mercedes-benz,clk-klasa,GNI,WIE,255223.0,1796.0,0,0,1,0,0,2.879760,1,0,0,독일,유럽,1.095273,3.152736,3.785779,4.354655,5.049856,2.250239,3.147165,3.440418,3.627069,4.866534,0.482426,3.089678,3.707577,4.296605,5.049856,0.955511,2.453588,2.826722,3.353407,5.049022,2.250239,3.147165,3.440418,3.627069,4.866534,1.118415,3.130263,3.746677,4.334280,5.049022,1.967112,3.152736,3.393501,3.590991,5.049022,2.402430,2.899306,3.339677,3.517711,3.629660,2.554899,3.253084,3.480932,3.660223,4.866534,1.175573,2.354152,2.721953,3.232779,4.631812,2.402430,2.899306,3.339677,3.517711,3.629660,2.554899,3.253084,3.480932,3.660223,4.866534,2.513656,3.174346,3.397524,3.516607,4.154185,2.554899,3.145882,3.392547,3.568935,3.629660
3,2006,2001,nissan,x-trail,EHX,WIE,238000.0,2184.0,0,1,0,0,0,2.901422,0,5,0,일본,아시아,1.050822,3.558201,3.923359,4.405499,5.049022,2.164472,4.368303,4.575844,4.761062,5.049022,0.482426,3.089678,3.707577,4.296605,5.049856,0.955511,2.341806,2.738903,3.095125,4.886356,2.164472,4.368303,4.575844,4.761062,5.049022,1.291984,3.477232,3.946231,4.321082,4.943569,1.291984,2.261556,2.613740,2.963725,3.393501,2.901422,4.478367,4.668990,4.761358,4.901936,2.164472,2.879654,2.963725,3.130247,3.393501,1.291984,2.193886,2.497726,2.827314,4.172385,2.901422,4.478367,4.668990,4.761358,4.901936,2.164472,2.879654,2.963725,3.130247,3.393501,1.291984,2.350294,2.501657,2.934804,3.201526,2.901422,2.968187,3.0

<br></br>

# EDA

In [30]:
# check_num_features = [col for col in num_features if col.find('_Q')<0]

# i=0
# for col in check_num_features:
#     i+=1
#     print('\n({}/{}) {}'.format(i,len(check_num_features),col))
#     plt.figure(figsize=(15,7))
#     sns.scatterplot(x=train_df3['가격'],y=train_df3[col])
#     plt.show()

<br></br>

# Add the Interaction Term

In [31]:
import warnings
from tqdm import trange

def get_abs_corr(x,y):
    return np.abs(np.corrcoef(x,y))[0,1]

class InteractionTerm:
    def __init__(self):
        pass
    
    def fit(self,data,num_features,corr_cutoff=0.7):
        warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
        
        d = data.copy()
        self.interaction_list = []
        total_cnt = 0
        add_cnt = 0
        for i in range(len(num_features)):
            for j in range(len(num_features)):
                if i>j:
                    total_cnt+=1
                    col_i = num_features[i]
                    col_j = num_features[j]
                    
                    # 상관계수가 cutoff보다 큰 경우에는 interaction을 생성하지 않음
                    if (get_abs_corr(d[col_i]*d[col_j],d[col_i])>=corr_cutoff) | (get_abs_corr(d[col_i]*d[col_j],d[col_j])>=corr_cutoff):
                        pass
                    else:
                        self.interaction_list.append(f'{col_i}*{col_j}')
                        add_cnt+=1
        print('> Total Interaction Term: {}'.format(total_cnt))
        print('> Added Interaction Term: {}'.format(add_cnt))
    
    def transform(self,data):
        d = data.copy()
        for interaction in self.interaction_list:
            col_i,col_j = interaction.split('*')
            d[interaction] = d[col_i]*d[col_j]
        return d
    
    def fit_transform(self,data,num_features,corr_cutoff=0.7):
        self.fit(data,num_features,corr_cutoff)
        return self.transform(data)

In [32]:
train_df4 = train_df3.copy()
test_df4  = test_df3.copy()

if CFG.INTERACTION:
    interaction_maker = InteractionTerm()
    interaction_maker.fit(
        data=train_df3,
        num_features=num_features,
        corr_cutoff=0.85,
    )
    train_df4 = interaction_maker.transform(train_df4)
    test_df4  = interaction_maker.transform(test_df4)

    type_resetor = TypeResetting()
    type_resetor.add_categorical_features(fe.new_cat_features)
    type_resetor.fit(train_df4)
    type_resetor.get_feature_type()

    train_df4 = type_resetor.transform(train_df4)
    test_df4  = type_resetor.transform(test_df4)

In [33]:
train_df3.shape,train_df4.shape

((57920, 89), (57920, 89))

<br></br>

# Feature Selection

In [34]:
# k=0
# for i in range(len(num_features)):
#     for j in range(len(num_features)):
#         if i>j:
#             col_i = num_features[i]
#             col_j = num_features[j]
#             corr = np.corrcoef(train_df4[col_i],train_df4[col_j])[0,1]
#             if corr>=0.7:
#                 k+=1
#                 print(k,col_i,col_j,corr)

In [35]:
def log_offset(x):
    if min(x)>0:
        offset = 0
    elif min(x)==0:
        offset = 1e-3
    else:
        offset = min(x)+1e-3
        print('minimum = {:.3f}'.format(min(x)))
    return np.log(x+offset)

<br></br>

## Categorical Features

In [36]:
# import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [37]:
check_cat_features = [col for col in cat_features if train_df4[col].nunique()<=100]

# (1) ANOVA를 해서 p-value가 0.05보다 높은 것들 확인
pvalue_list = []
for col in tqdm(check_cat_features):
    d = train_df4[[col,'가격']].rename(columns={col:'feature'})
    
    model = ols(f'가격 ~ C(feature)',data=d).fit()
    pvalue = anova_lm(model).values[0][-1]
    pvalue_list.append([col,pvalue])
    
pvalue_df = pd.DataFrame(pvalue_list,columns=['feature','pvalue'])\
    .sort_values('pvalue',ascending=False)
# pvalue_df[pvalue_df.pvalue>=alpha].round(4)

100%|██████████| 9/9 [00:05<00:00,  1.64it/s]


In [38]:
# (2) (1)에서 유의하지않은 feature들은 log적용 후에도 유의하지 않으면 제외
pvalue_list2 = []
unsignificant_features = pvalue_df[pvalue_df.pvalue>CFG.FS_ALPHA].feature.tolist()
for col in tqdm(unsignificant_features):
    d = train_df4[[col,'target']].rename(columns={col:'feature'})
    d['feature'] = log_offset(d['feature'])
    
    model = ols(f'feature ~ C(target)',data=d).fit()
    pvalue = anova_lm(model).values[0][-1]
    pvalue_list2.append([col,pvalue])
    
pvalue_df2 = pd.DataFrame(pvalue_list2,columns=['feature','pvalue'])\
    .sort_values('pvalue',ascending=False)

0it [00:00, ?it/s]


In [39]:
delete_features = pvalue_df2[pvalue_df2.pvalue> CFG.FS_ALPHA].feature.tolist()
log_features    = pvalue_df2[pvalue_df2.pvalue<=CFG.FS_ALPHA].feature.tolist()
print('> delete_features')
print('  - length : {}'.format(len(delete_features)))
print('  - feature_name : {}'.format(delete_features))
print('')
print('> log_features')
print('  - length : {}'.format(len(log_features)))
print('  - feature_name : {}'.format(log_features))

train_df5 = train_df4.copy()
train_df5.drop(delete_features,axis=1,inplace=True)
for col in log_features:
    train_df5[col] = log_offset(train_df5[col])
    
test_df5 = test_df4.copy()
test_df5.drop(delete_features,axis=1,inplace=True)
for col in log_features:
    test_df5[col] = log_offset(test_df5[col])

> delete_features
  - length : 0
  - feature_name : []

> log_features
  - length : 0
  - feature_name : []


<br>

## Numerical Features

In [40]:
import scipy

In [41]:
# (1) corr test를 해서 p-value가 0.05보다 높은 것들 확인
pvalue_list = []
for col in tqdm(num_features):
    corr,pvalue = scipy.stats.pearsonr(train_df5['가격'],train_df5[col])
    pvalue_list.append([col,pvalue])
pvalue_df = pd.DataFrame(pvalue_list,columns=['feature','pvalue'])\
    .sort_values('pvalue',ascending=False)
# pvalue_df.round(4).head()

100%|██████████| 72/72 [00:01<00:00, 65.99it/s]


In [42]:
# (2) (1)에서 유의하지않은 feature들은 log적용 후에도 유의하지 않으면 제외
pvalue_list2 = []
unsignificant_features = pvalue_df[pvalue_df.pvalue>CFG.FS_ALPHA].feature.tolist()
for col in tqdm(unsignificant_features):
    corr,pvalue = scipy.stats.pearsonr(train_df5['가격'],log_offset(train_df5[col]))
    pvalue_list2.append([col,pvalue])
pvalue_df2 = pd.DataFrame(pvalue_list2,columns=['feature','pvalue'])\
    .sort_values('pvalue',ascending=False)
# pvalue_df2.round(4).head()

0it [00:00, ?it/s]


In [43]:
delete_features = pvalue_df2[pvalue_df2.pvalue> CFG.FS_ALPHA].feature.tolist()
log_features    = pvalue_df2[pvalue_df2.pvalue<=CFG.FS_ALPHA].feature.tolist()
print('> delete_features')
print('  - length : {}'.format(len(delete_features)))
print('  - feature_name : {}'.format(delete_features))
print('')
print('> log_features')
print('  - length : {}'.format(len(log_features)))
print('  - feature_name : {}'.format(log_features))

train_df6 = train_df5.copy()
train_df6.drop(delete_features,axis=1,inplace=True)
for col in log_features:
    train_df6[col] = log_offset(train_df6[col])
    
test_df6 = test_df5.copy()
test_df6.drop(delete_features,axis=1,inplace=True)
for col in log_features:
    test_df6[col] = log_offset(test_df6[col])

> delete_features
  - length : 0
  - feature_name : []

> log_features
  - length : 0
  - feature_name : []


<br></br>

# Make Segment

In [44]:
def make_segment(data,segment: list):
    d = data.copy()
    d['segment'] = d[segment].apply(lambda x: '___'.join(x),axis=1)
    d.drop(columns=segment,inplace=True)
    return d

In [45]:
segment = ['브랜드','판매구역']
train_df7 = make_segment(train_df6,segment)
test_df7  = make_segment(test_df6 ,segment)

In [46]:
test_only = list(set(test_df7.segment.unique())-set(train_df7.segment.unique()))
assert len(test_only)==0, \
    "Segment exists only in the test set ({})".format(len(test_only))

In [47]:
train_only = list(set(train_df7['segment'].unique())-set(test_df7['segment'].unique()))

n_asis = len(train_df7)
n_tobe = len(train_df7[~train_df7.segment.isin(train_only)])
train_df7 = train_df7[~train_df7.segment.isin(train_only)]
print('> Train에만 존재하는 Segment 제거')
print(' - 데이터수 : {:,} -> {:,}'.format(n_asis,n_tobe))
print(' - 세그먼트수 : {:,}'.format(train_df7['segment'].nunique()))

> Train에만 존재하는 Segment 제거
 - 데이터수 : 57,920 -> 57,871
 - 세그먼트수 : 320


In [48]:
vc = train_df7['segment'].value_counts().sort_values()
display(vc.head())
print('...')
display(vc.tail())

segment
mitsubishi___WAR    7
volkswagen___MOR    8
skoda___MOR         8
peugeot___OPO       9
fiat___POL          9
Name: count, dtype: int64

...


segment
opel___SLA    1025
ford___MAZ    1040
bmw___MAZ     1043
audi___MAZ    1107
opel___MAZ    1203
Name: count, dtype: int64

In [49]:
type_resetor = TypeResetting()
type_resetor.add_categorical_features(fe.new_cat_features)
type_resetor.add_segment_features(segment)
type_resetor.fit(train_df7)
type_resetor.get_feature_type()

train_df7 = type_resetor.transform(train_df7)
test_df7  = type_resetor.transform(test_df7)

In [50]:
cat_features

['차량모델명',
 '판매도시',
 '생산년도',
 '모델출시년도',
 '출시년도생산여부',
 '출시이후생산년수',
 '출시이전생산여부',
 '브랜드국적',
 '브랜드대륙명']

In [51]:
print(train_df7.shape)
train_df7.head()

(57871, 88)


,생산년도,모델출시년도,차량모델명,판매도시,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격,출시년도생산여부,출시이후생산년수,출시이전생산여부,브랜드국적,브랜드대륙명,브랜드_Q0,브랜드_Q25,브랜드_Q50,브랜드_Q75,브랜드_Q100,차량모델명_Q0,차량모델명_Q25,차량모델명_Q50,차량모델명_Q75,차량모델명_Q100,판매구역_Q0,판매구역_Q25,판매구역_Q50,판매구역_Q75,판매구역_Q100,모델출시년도_Q0,모델출시년도_Q25,모델출시년도_Q50,모델출시년도_Q75,모델출시년도_Q100,브랜드_차량모델명_Q0,브랜드_차량모델명_Q25,브랜드_차량모델명_Q50,브랜드_차량모델명_Q75,브랜드_차량모델명_Q100,브랜드_판매구역_Q0,브랜드_판매구역_Q25,브랜드_판매구역_Q50,브랜드_판매구역_Q75,브랜드_판매구역_Q100,브랜드_모델출시년도_Q0,브랜드_모델출시년도_Q25,브랜드_모델출시년도_Q50,브랜드_모델출시년도_Q75,브랜드_모델출시년도_Q100,차량모델명_판매구역_Q0,차량모델명_판매구역_Q25,차량모델명_판매구역_Q50,차량모델명_판매구역_Q75,차량모델명_판매구역_Q100,차량모델명_모델출시년도_Q0,차량모델명_모델출시년도_Q25,차량모델명_모델출시년도_Q50,차량모델명_모델출시년도_Q75,차량모델명_모델출시년도_Q100,판매구역_모델출시년도_Q0,판매구역_모델출시년도_Q25,판매구역_모델출시년도_Q50,판매구역_모델출시년도_Q75,판매구역_모델출시년도_Q100,브랜드_차량모델명_판매구역_Q0,브랜드_차량모델명_판매구역_Q25,브랜드_차량모델명_판매구역_Q50,브랜드_차량모델명_판매구역_Q75,브랜드_차량모델명_판매구역_Q100,브랜드_차량모델명_모델출시년도_Q0,브랜드_차량모델명_모델출시년도_Q25,브랜드_차량모델명_모델출시년도_Q50,브랜드_차량모델명_모델출시년도_Q75,브랜드_차량모델명_모델출시년도_Q100,브랜드_판매구역_모델출시년도_Q0,브랜드_판매구역_모델출시년도_Q25,브랜드_판매구역_모델출시년도_Q50,브랜드_판매구역_모델출시년도_Q75,브랜드_판매구역_모델출시년도_Q100,차량모델명_판매구역_모델출시년도_Q0,차량모델명_판매구역_모델출시년도_Q25,차량모델명_판매구역_모델출시년도_Q50,차량모델명_판매구역_모델출시년도_Q75,차량모델명_판매구역_모델출시년도_Q100,segment
0,2018,2014,fabia,KAT,85231.0,999.0,0,0,1,0,0,3.946231,0,4,0,체코,유럽,0.727549,3.393501,4.057853,4.509072,5.049856,0.792993,2.933059,3.712352,4.065687,4.830312,0.262364,3.206803,3.773910,4.355041,5.049856,0.157004,4.019486,4.355041,4.643140,5.049856,0.792993,2.933059,3.712352,4.065687,4.830312,1.137833,3.477232,4.021236,4.525044,5.049856,2.554899,3.843102,4.011868,4.172385,4.772124,1.137833,2.995955,3.814851,4.046554,4.741361,2.554899,3.843102,4.011868,4.172385,4.772124,3.287655,3.984523,4.322807,4.617790,5.049856,1.137833,2.995955,3.814851,4.046554,4.741361,2.554899,3.843102,4.011868,4.172385,4.772124,3.520461,3.864403,3.962369,4.120439,4.639378,3.520461,3.864403,3.962369,4.120439,4.639378,skoda___SLA
1,2010,2006,auris,RKO,135000.0,1598.0,0,0,1,0,0,3.724970,0,4,0,일본,아시아,0.157004,3.353407,4.000034,4.435212,5.049856,2.687167,3.594569,4.112512,4.387075,4.761062,1.095273,3.113071,3.520461,4.066802,5.049856,1.358409,3.089678,3.348851,3.785779,5.049856,2.687167,3.594569,4.112512,4.387075,4.761062,2.134166,3.341912,3.843102,4.286479,5.049022,2.687167,3.253084,3.501043,3.785779,4.450503,2.865054,3.356793,3.986240,4.276820,4.691990,2.687167,3.201526,3.353407,3.594569,4.173156,2.277267,3.127414,3.353407,3.782821,4.826712,2.865054,3.356793,3.986240,4.276820,4.691990,2.687167,3.201526,3.353407,3.594569,4.173156,2.865054,3.318510,3.606977,3.932255,4.450503,2.865054,3.144356,3.341912,3.632507,4.021236,toyota___SWI
2,2002,2002,clk-klasa,GNI,255223.0,1796.0,0,0,1,0,0,2.879760,1,0,0,독일,유럽,1.095273,3.152736,3.785779,4.354655,5.049856,2.250239,3.147165,3.440418,3.627069,4.866534,0.482426,3.089678,3.707577,4.296605,5.049856,0.955511,2.453588,2.826722,3.353407,5.049022,2.250239,3.147165,3.440418,3.627069,4.866534,1.118415,3.130263,3.746677,4.334280,5.049022,1.967112,3.152736,3.393501,3.590991,5.049022,2.402430,2.899306,3.339677,3.517711,3.629660,2.554899,3.253084,3.480932,3.660223,4.866534,1.175573,2.354152,2.721953,3.232779,4.631812,2.402430,2.899306,3.339677,3.517711,3.629660,2.554899,3.253084,3.480932,3.660223,4.866534,2.513656,3.174346,3.397524,3.516607,4.154185,2.554899,3.145882,3.392547,3.568935,3.629660,mercedes-benz___WIE
3,2006,2001,x-trail,EHX,238000.0,2184.0,0,1,0,0,0,2.901422,0,5,0,일본,아시아,1.050822,3.558201,3.923359,4.405499,5.049022,2.164472,4.368303,4.575844,4.761062,5.049022,0.482426,3.089678,3.707577,4.296605,5.049856,0.955511,2.341806,2.738903,3.095125,4.886356,2.164472,4.368303,4.575844,4.761062,5.049022,1.291984,3.477232,3.946231,4.321082,4.943569,1.291984,2.261556,2.613740,2.963725,3.393501,2.901422,4.478367,4.668990,4.761358,4.901936,2.164472,2.879654,2.963725,3.130247,3.393501,1.291984,2.193886,2.497726,2.827314,4.172385,2.901422,4.478367,4.668990,4.761358,4.901936,2.164472,2.879654,2.963725,3.130247,3.393501,1.291984,2.350294,2.501657,2.934804,3.201526,2.901422,2.968187,3.034953,

<br></br>

# Modeling

In [52]:
import os
def mkdir(paths):
    if type(paths)==str:
        paths = [paths]
    for path in paths:
        if not os.path.isdir(path):
            print('> Create Folder: {}'.format(path))
            os.mkdir(path)

In [53]:
## dummy_features는 한가지만 속함
# X[dummy_features].apply(lambda x: np.sum(x),axis=1).value_counts()

def add_fuel_type(data,dummy_features):
    d = data.copy()
    d['fuel_type'] = d[dummy_features].apply(
        lambda x: dummy_features[np.where(x==1)[0][0]],axis=1)
    d.drop(columns=dummy_features,inplace=True)
    return d

In [54]:
cat_path = 'segment_catboost2'
weiens_path = 'segment_weightedensemble2'

mkdir(f'./model_checkpoints')
mkdir(f'./model_checkpoints/{cat_path}')
mkdir(f'./model_checkpoints/{weiens_path}')

<br>

## CatBoost
- public score : 

In [55]:
gc.collect()

0

In [56]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor, Pool

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [57]:
train_fn = train_df7.copy()
test_fn  = test_df7 .copy()

In [61]:
%%time
# 30분

X = train_fn.drop(target_feature,axis=1)
y = train_fn[target_feature]

X = add_fuel_type(X,dummy_features)
new_cat_features = cat_features + ['fuel_type']

segment_list = X['segment'].unique()

models = {}
feature_info = {}
scores = []
pbar = tqdm(segment_list)

s_i = 0
for segment in pbar:
    s_i+=1
    
    # segment에 해당하는 데이터추출
    _X = X[X.segment==segment].drop('segment',axis=1)
    _y = y[X.segment==segment]
    
    # kfold
    kf = KFold(n_splits=CFG.N_SPLITS,random_state=1000*s_i+CFG.SEED,shuffle=True)
    
    # unique인 컬럼 제외
    unique_info = _X.nunique()
    unique_cols = unique_info[unique_info==1].index.tolist()
    if len(unique_cols)>0:
        _X = _X.drop(unique_cols,axis=1)
        
    # categorical feature에서 unique인 컬럼을 제외
    fixed_cat_features = [col for col in new_cat_features if col in _X.columns]
    
    _models = []
    _scores = []
    k=0
    for tr_idx,val_idx in kf.split(_X,_y):
        k+=1
        
        # kfold dataset
        X_tr, X_va = _X.iloc[tr_idx], _X.iloc[val_idx]
        y_tr, y_va = _y.iloc[tr_idx], _y.iloc[val_idx]

        # progress
        progress = 'Segment: {}, Size: {}, KFold: [{}/{}]'\
            .format(segment,len(_X),k,CFG.N_SPLITS)
        pbar.set_description(progress)

        # dataset
        train_dataset = Pool(X_tr,y_tr,cat_features=fixed_cat_features)
        valid_dataset = Pool(X_va,y_va,cat_features=fixed_cat_features)

        # define the model
        model = CatBoostRegressor(
            loss_function='MAE',
            random_state=CFG.SEED,
            iterations=CFG.EPOCHS,
            learning_rate=CFG.LR,
            #max_depth=7,
            #max_bin=64,
            allow_writing_files=False,
        )

        # fit the model
        model.fit(
            train_dataset,
            eval_set=valid_dataset,
            early_stopping_rounds=CFG.ES,
            verbose=0,
            #metric_period=CFG.EPOCHS//5,
        )

        # save the model
        model.save_model(f'./model_checkpoints/{cat_path}/{segment}_k{k}.cbm')

        # calculate the score
        y_pred = model.predict(valid_dataset).flatten()
        y_true = y_va.values
        if CFG.TARGET_TRANSFORMATION:
            y_pred = np.exp(y_pred)
            y_true = np.exp(y_true)
        score = mean_absolute_error(y_true=y_true,y_pred=y_pred)
        
        # append inner loop
        _models.append(model)
        _scores.append([segment,k,len(X_tr),len(X_va),score])

    # append outer loop
    models[segment] = _models
    scores.append(_scores)
    feature_info[segment] = {'cat_features':fixed_cat_features,'features':_X.columns.tolist()}
    
    # score report
    mean_score_report = pd.Series(np.array(_scores)[:,-1]).astype(float).mean()
    print("MAE's for {}-Fold: {}".format(CFG.N_SPLITS,np.array(pd.Series(np.array(_scores)[:,-1]).astype(float).values)))
    print("Mean of MAE's for {}-Fold: {:.4f}".format(CFG.N_SPLITS,mean_score_report))

Segment: toyota___SWI, Size: 103, KFold: [1/5]:   0%|          | 1/320 [00:26<2:20:38, 26.45s/it]

MAE's for 5-Fold: [8.08391617 7.58578684 9.0635417  7.65549564 7.31439177]
Mean of MAE's for 5-Fold: 7.9406


Segment: mercedes-benz___WIE, Size: 445, KFold: [1/5]:   1%|          | 2/320 [00:45<1:57:46, 22.22s/it]

MAE's for 5-Fold: [10.13881661  7.70861585  7.87973072  8.19800516  8.91398066]
Mean of MAE's for 5-Fold: 8.5678


Segment: nissan___WIE, Size: 217, KFold: [1/5]:   1%|          | 3/320 [00:55<1:27:23, 16.54s/it]       

...

MAE's for 5-Fold: [14.90500509 10.51756768 20.99129768 22.08864721  5.79533083]
Mean of MAE's for 5-Fold: 14.8596


Segment: fiat___POL, Size: 9, KFold: [1/5]: 100%|█████████▉| 319/320 [55:20<00:03,  4.00s/it]      

MAE's for 5-Fold: [ 3.68885593 11.6059256  34.65900314 11.40700042  5.89838858]
Mean of MAE's for 5-Fold: 13.4518


Segment: fiat___POL, Size: 9, KFold: [5/5]: 100%|██████████| 320/320 [55:25<00:00, 10.39s/it]

MAE's for 5-Fold: [14.52418858  8.45021522  5.56824535  5.25565355 31.59611022]
Mean of MAE's for 5-Fold: 13.0789
CPU times: user 2h 4min 52s, sys: 36min 2s, total: 2h 40min 54s
Wall time: 55min 29s


In [62]:
import pickle
with open(f'./model_checkpoints/{cat_path}/models.pkl', 'wb') as f:
	pickle.dump(models, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'./model_checkpoints/{cat_path}/feature_info.pkl', 'wb') as f:
	pickle.dump(feature_info, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'./model_checkpoints/{cat_path}/scores.pkl', 'wb') as f:
	pickle.dump(scores, f, protocol=pickle.HIGHEST_PROTOCOL)

In [76]:
score_df = pd.DataFrame(
    np.array(scores).reshape(len(scores)*5,5),
    columns=['segment','k','n_tr','n_val','score']
)

score_df.sort_values(['segment','k']).head(10)

,segment,k,n_tr,n_val,score
150,audi___DOL,1,375,94,7.4205131861914655
151,audi___DOL,2,375,94,5.874012023013503
152,audi___DOL,3,375,94,8.379563016695236
153,audi___DOL,4,375,94,5.92707977356499
154,audi___DOL,5,376,93,8.840820570437545
735,audi___KUJ,1,192,48,6.027368821159008
736,audi___KUJ,2,192,48,7.091968349057946
737,audi___KUJ,3,192,48,6.892028525794731
738,audi___KUJ,4,192,48,7.201172892550677
739,audi___KUJ,5,192,48,7.005191109486241


In [101]:
# inference
X = train_fn.drop(target_feature,axis=1)
X = add_fuel_type(X,dummy_features)
y = train_fn[target_feature]

X_test = add_fuel_type(test_fn,dummy_features)

segment_list = X['segment'].unique()

tr_pred_list = []
te_pred_list = []
for segment in tqdm(segment_list):
    ## data load
    # (1) train
    train_data = X[X.segment==segment][feature_info[segment]['features']]
    train_dataset = Pool(train_data,cat_features=feature_info[segment]['cat_features'])
    # (2) test
    test_data = X_test[X_test.segment==segment][feature_info[segment]['features']]
    test_dataset = Pool(test_data,cat_features=feature_info[segment]['cat_features'])
    
    ## model
    kfold_models = models[segment]
    kfold_scores = score_df[score_df.segment==segment].sort_values('k').score.astype(float).values
    kfold_weights = 1/kfold_scores
    kfold_weights /= sum(kfold_weights)

    ## prediction
    # (1) train
    tr_pred_df = pd.DataFrame({
        'segment':segment,
        'true':y[X.segment==segment].values.flatten(),
        'pred':np.mean([model.predict(train_dataset) for model in kfold_models],axis=0),
        #'pred':np.sum([weight*model.predict(train_dataset) for weight,model in zip(kfold_weights,kfold_models)],axis=0),
    })
    tr_pred_df.index = train_data.index
    # (2) test
    te_pred_df = pd.DataFrame({
        'segment':segment,
        'pred':np.mean([model.predict(test_dataset) for model in kfold_models],axis=0),
        #'pred':np.sum([weight*model.predict(test_dataset) for weight,model in zip(kfold_weights,kfold_models)],axis=0),
    })
    te_pred_df.index = test_data.index
    
    ## Target Transformation
    if CFG.TARGET_TRANSFORMATION:
        tr_pred_df['true'] = np.exp(tr_pred_df['true'])
        tr_pred_df['pred'] = np.exp(tr_pred_df['pred'])
        te_pred_df['pred'] = np.exp(te_pred_df['pred'])
    
    ## append
    tr_pred_list.append(tr_pred_df)
    te_pred_list.append(te_pred_df)

100%|██████████| 320/320 [00:07<00:00, 43.06it/s]


In [102]:
# train
tr_pred_df = pd.concat(tr_pred_list,axis=0).sort_index()
mean_absolute_error(y_true=tr_pred_df.true,y_pred=tr_pred_df.pred)

5.012146835330443

In [103]:
te_pred_df = pd.concat(te_pred_list,axis=0).sort_index()
te_pred_df.head()

,segment,pred
0,mazda___WIE,74.328589
1,ford___POD,27.552190
2,volkswagen___LOD,86.825672
3,renault___SLA,115.033888
4,volvo___POM,57.511411


In [100]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['가격'] = te_pred_df.pred.values
submit.to_csv('./out/9_catboost_segment_브랜드&판매구역_kfold.csv',index=False)

- weighted sum으로 바꾸기
- cat, xgb, lgb 비교해서 필요없는거 하나 빼기?

<br>

## Weighted Ensemble
- public score : 6.2489713335

In [ ]:
import pandas as pd
import warnings

class OneHotEncoder:
    def __init__(self):
        pass
    
    def fit(self,data,columns):
        self.transform_list = []
        for col in columns:
            for i,value in enumerate(sorted(data[col].unique())):
                if i>0:
                    self.transform_list.append([col,value])
        
    def transform(self,data):
        warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
        new_data = data.copy()
        for col,value in self.transform_list:
            new_data[f'{col}_{value}'] = np.where(new_data[col]==value,1,0)
        drop_columns = pd.unique(np.array(self.transform_list)[:,0])
        new_data.drop(columns=drop_columns,inplace=True)
        return new_data

In [ ]:
from copy import deepcopy
from sklearn.base import BaseEstimator, RegressorMixin
from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
import warnings
import time
import pickle

class WeightedEnsembleRegressor(BaseEstimator, RegressorMixin):
    def __init__(self,weight=['equal','balanced']):
        super().__init__()
        
        assert weight in ['equal','balanced'], \
            "weight must be one of ['equal','balanced']"
        self.weight = weight
        self._get_regressors()
    
    def _get_regressors(self):
        max_depth = 10
        n_jobs = -1
        
        params_catboost = {
            'random_state':CFG.SEED,
            'early_stopping_rounds' : CFG.ES,
            'learning_rate' : CFG.LR,
            'iterations' : CFG.EPOCHS,
            'loss_function': 'MAE',
            'grow_policy' : 'Lossguide', # 'SymmetricTree','Depthwise'
            'use_best_model' : True,
            'allow_writing_files' : False,
            'verbose' : 0,
            'max_depth': max_depth,
            'l2_leaf_reg' : 1,
        }
    
        params_xgboost = {
            'random_state':CFG.SEED,
            'early_stopping_rounds' : CFG.XGB_ES,
            'learning_rate' : CFG.XGB_LR,
            'n_estimators' : CFG.XGB_EPOCHS,
            'objective': 'reg:absoluteerror',
            'verbosity' : 0,
            'max_depth': max_depth,
            'n_jobs' : n_jobs,
        }
    
        params_lgb = {
            'objective': 'regression',
            'random_state':CFG.SEED,
            'early_stopping_round' : CFG.ES,
            'learning_rate' : CFG.LR,
            'n_estimators' : CFG.EPOCHS,
            'metric': 'mean_absolute_error',
            'verbosity' : -1,
            'max_depth': max_depth,
            'n_jobs' : n_jobs,
        }
        
        self.regressors = [
            CatBoostRegressor(**params_catboost),
            XGBRegressor(**params_xgboost),
            LGBMRegressor(**params_lgb),
        ]
        self.regressors_name = ['CatBoost','XGBoost','LightGBM']
    
    def fit(self,X,y,eval_set,oh_set,cat_features,verbose=1):
        assert len(eval_set)==1, \
            "eval_set length must be 1. len(eval_set)={}".format(len(eval_set))
        assert len(oh_set)==1, \
            "oh_set length must be 1. len(oh_set)={}".format(len(oh_set))
        X_val, y_val = eval_set[0]
        X_oh, X_val_oh = oh_set[0]
        
        self.cat_features = cat_features
        self.weights = []
        self.fitting_elapsed = []
        if verbose:
            pbar = tqdm(zip(self.regressors_name,self.regressors),total=len(self.regressors))
        else:
            pbar = zip(self.regressors_name,self.regressors)
        for regressor_name,regressor in pbar:
            s = time.time()
            if verbose:
                pbar.set_description(name)
            if regressor_name=='CatBoost':
                train_dataset = Pool(X,y,cat_features=cat_features)
                val_dataset   = Pool(X_val,y_val,cat_features=cat_features)
                regressor.fit(
                    train_dataset,
                    eval_set=val_dataset,
                    #metric_period=CFG.EPOCHS//5,
                )
                val_pred = regressor.predict(val_dataset)
            elif regressor_name=='XGBoost':
                regressor.fit(
                    X_oh,y,
                    eval_set=[(X_val_oh,y_val)],
                    verbose=0,
                )
                val_pred = regressor.predict(X_val_oh)
            elif regressor_name=='LightGBM':
                warnings.filterwarnings("ignore", category=UserWarning)
                X_tmp = X.copy()
                X_val_tmp = X_val.copy()
                for col in cat_features:
                    X_tmp[col]     = X_tmp[col]    .astype('category')
                    X_val_tmp[col] = X_val_tmp[col].astype('category')
                regressor.fit(
                    X_tmp,y,
                    eval_set=[(X_val_tmp,y_val)],
                    verbose=-1,
                )
                val_pred = regressor.predict(X_val_tmp)
            else:
                raise ValueError('Unknown Regressor: {}'.format(regressor_name))
                
            score = mean_absolute_error(y_pred=val_pred,y_true=y_val)
            e = time.time()
            
            self.weights.append(1/score)
            self.fitting_elapsed.append(e-s)
        
        if self.weight=='equal':
            self.weights = np.array([1.0 for _ in self.regressors])
        self.weights /= sum(self.weights)
                
    def predict(self,X,X_oh):
        assert len(X)==len(X_oh), \
            "X and X_oh must be same length"
        
        pred_list = []
        for regressor_name,regressor in zip(self.regressors_name,self.regressors):
            if regressor_name=='CatBoost':
                dataset = Pool(X,cat_features=self.cat_features)
            elif regressor_name=='XGBoost':
                dataset = X_oh.copy()
            elif regressor_name=='LightGBM':
                dataset = X.copy()
                for col in self.cat_features:
                    dataset[col] = dataset[col].astype('category')
            else:
                raise ValueError('Unknown Regressor: {}'.format(regressor_name))
            
            y_pred = regressor.predict(dataset)
            y_pred = np.array(y_pred).flatten()
            pred_list.append(y_pred)
            
        final_pred = np.zeros(len(X))
        for pred,weight in zip(pred_list,self.weights):
            final_pred += np.array(pred)*weight
            
        return final_pred
    
    def save_model(self,path):
        save_dict = {
            'cat_features' : self.cat_features,
            'weights' : self.weights,
            'fitting_elapsed' : self.fitting_elapsed,
            'regressors' : self.regressors,
        }
        with open(path, 'wb') as f:
            pickle.dump(save_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
            
    def load_model(self,path):
        with open(path, 'rb') as f:
            save_dict = pickle.load(f)
            self.cat_features = save_dict['cat_features']
            self.weights = save_dict['weights']
            self.fitting_elapsed = save_dict['fitting_elapsed']
            self.regressors = save_dict['regressors']

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
gc.collect()

In [ ]:
train_fn = train_df7.copy()
test_fn  = test_df7 .copy()

In [ ]:
%%time
# 6시간

X = train_fn.drop(target_feature,axis=1)
y = train_fn[target_feature]

ohe = OneHotEncoder()
ohe.fit(X,cat_features)
X_oh = ohe.transform(X)

segment_list = X['segment'].unique()

models = {}
feature_info = {}
scores = []
pbar = tqdm(segment_list)

s_i = 0
for segment in pbar:
    s_i+=1
    
    # segment에 해당하는 데이터추출
    _X    = X   [X   .segment==segment].drop('segment',axis=1)
    _X_oh = X_oh[X_oh.segment==segment].drop('segment',axis=1)
    _y    = y   [X   .segment==segment]
    
    # kfold
    kf = KFold(n_splits=CFG.N_SPLITS,random_state=1000*s_i+CFG.SEED,shuffle=True)
    
    # unique인 컬럼 제외
    # (1) X
    unique_info = _X.nunique()
    unique_cols = unique_info[unique_info==1].index.tolist()
    if len(unique_cols)>0:
        _X = _X.drop(unique_cols,axis=1)
    # (2) X_oh
    unique_info = _X_oh.nunique()
    unique_cols = unique_info[unique_info==1].index.tolist()
    if len(unique_cols)>0:
        _X_oh = _X_oh.drop(unique_cols,axis=1)
        
    # categorical feature에서 unique인 컬럼을 제외
    fixed_cat_features = [col for col in cat_features if col in _X.columns]
    
    _models = []
    _scores = []
    k=0
    for tr_idx,val_idx in kf.split(_X,_y):
        k+=1
        
        # kfold dataset
        X_tr   , X_va    = _X   .iloc[tr_idx], _X   .iloc[val_idx]
        X_tr_oh, X_va_oh = _X_oh.iloc[tr_idx], _X_oh.iloc[val_idx]
        y_tr   , y_va    = _y   .iloc[tr_idx], _y   .iloc[val_idx]

        # progress
        progress = 'Segment: {}, Length: Train({}), Validation({}), KFold: {}/{}'\
            .format(segment,len(X_tr),len(X_va),k,CFG.N_SPLITS)
        pbar.set_description(progress)

        # define the model
        ensemble_model = WeightedEnsembleRegressor(weight='equal')

        # fit the model
        ensemble_model.fit(
            X_tr,y_tr,
            eval_set=[(X_va,y_va)],
            oh_set=[(X_tr_oh,X_va_oh)],
            cat_features=fixed_cat_features,
            verbose=0,
        )

        # save the model
        ensemble_model.save_model(f'./model_checkpoints/{weiens_path}/{segment}_k{k}.pickle')

        # calculate the score
        y_pred = ensemble_model.predict(X_va,X_va_oh).flatten()
        y_true = y_va.values
        score = mean_absolute_error(y_true=y_true,y_pred=y_pred)
        
        # append inner loop
        _models.append(ensemble_model)
        _scores.append([segment,k,len(X_tr),len(X_va),score])

    # append outer loop
    models[segment] = _models
    scores.append(_scores)
    feature_info[segment] = {
        'cat_features':fixed_cat_features,
        'features':_X.columns.tolist(),
        'oh_features':_X_oh.columns.tolist(),
    }

In [ ]:
import pickle
with open(f'./model_checkpoints/{weiens_path}/models.pkl', 'wb') as f:
	pickle.dump(models, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'./model_checkpoints/{weiens_path}/feature_info.pkl', 'wb') as f:
	pickle.dump(feature_info, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'./model_checkpoints/{weiens_path}/scores.pkl', 'wb') as f:
	pickle.dump(scores, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# import pickle
# with open(f'./model_checkpoints/{weiens_path}/models.pkl', 'rb') as f:
# 	models = pickle.load(f)
# with open(f'./model_checkpoints/{weiens_path}/feature_info.pkl', 'rb') as f:
# 	feature_info = pickle.load(f)
# with open(f'./model_checkpoints/{weiens_path}/scores.pkl', 'rb') as f:
# 	scores = pickle.load(f)

In [ ]:
# pd.DataFrame(
#     np.array(scores).reshape(100,5),
#     columns=['segment','k','n_tr','n_val','score']
# ).sort_values(['segment','k'])

In [ ]:
# inference
X = train_fn.drop(target_feature,axis=1)
y = train_fn[target_feature]

X_test = test_fn.copy()

ohe = OneHotEncoder()
ohe.fit(X,cat_features)
X_oh = ohe.transform(X)
X_test_oh = ohe.transform(X_test)

segment_list = X['segment'].unique()

tr_pred_list = []
te_pred_list = []
for segment in tqdm(segment_list):
    ## data load
    # (1) train
    train_data    = X   [X   .segment==segment][feature_info[segment]['features']]
    train_data_oh = X_oh[X_oh.segment==segment][feature_info[segment]['oh_features']]
    # (2) test
    test_data     = X_test   [X_test   .segment==segment][feature_info[segment]['features']]
    test_data_oh  = X_test_oh[X_test_oh.segment==segment][feature_info[segment]['oh_features']]
    
    ## model
    kfold_models = models[segment]
    
    ## prediction
    # (1) train
    tr_pred_df = pd.DataFrame({
        'true':y[X.segment==segment].values.flatten(),
        'pred':np.mean([model.predict(train_data,train_data_oh) for model in kfold_models],axis=0),
    })
    tr_pred_df.index = train_data.index
    # (2) test
    te_pred_df = pd.DataFrame({
        'pred':np.mean([model.predict(test_data,test_data_oh) for model in kfold_models],axis=0),
    })
    te_pred_df.index = test_data.index
    
    ## append
    tr_pred_list.append(tr_pred_df)
    te_pred_list.append(te_pred_df)

In [ ]:
# train
tr_pred_df = pd.concat(tr_pred_list,axis=0).sort_index()
mean_absolute_error(y_true=tr_pred_df.true,y_pred=tr_pred_df.pred)

In [ ]:
# def abline(intercept,slope,**kwargs):
#     axes = plt.gca()
#     x_vals = np.array(axes.get_xlim())
#     y_vals = intercept + slope * x_vals
#     plt.plot(x_vals, y_vals, '--',**kwargs)

# offset = 0.05
# min_value = min(tr_pred_df.true.min(),tr_pred_df.pred.min())*(1-offset)
# max_value = min(tr_pred_df.true.max(),tr_pred_df.pred.max())*(1+offset)

# plt.figure(figsize=(15,7))
# sns.scatterplot(x=tr_pred_df.true,y=tr_pred_df.pred)
# plt.xlim(min_value,max_value)
# plt.ylim(min_value,max_value)
# abline(0,1,color='red',linestyle='--')
# plt.show()

In [ ]:
te_pred_df = pd.concat(te_pred_list,axis=0).sort_index()
te_pred_df.head()

In [ ]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['가격'] = te_pred_df.pred.values
submit.to_csv('./out/10_ensemble_segment_브랜드&판매구역_kfold.csv',index=False)

<br>

## 참조 pycaret

In [ ]:
# from pycaret import regression

In [ ]:
# %%time

# data = train_fn[train_fn.segment==segment_list[0]]
# print(len(data))

# regression.setup(data=data,target='가격',remove_outliers=True,verbose=True)
# best = regression.compare_models(n_select=5,fold=5)